In [1]:
from datalake_tools.extract import connection # biblioteca para conexão no Data Lake
import pandas as pd # biblioteca para manipulação de dados tabulados
import numpy as np # biblioteca com funções matemáticas, lógicas, etc
from datetime import date, datetime # biblioteca com funções de data
from dateutil.relativedelta import relativedelta # função para cálculo de diferenças de datas
pd.set_option("display.max_columns", None)

In [2]:
df_contratos= pd.read_excel(r"C:\Users\julia.villela\Documents\Analista de Dados Sênior - Case.xlsx", sheet_name="Contratos")
df_consumo = pd.read_excel(r"C:\Users\julia.villela\Documents\Analista de Dados Sênior - Case.xlsx", sheet_name="Consumo")
df_clientes = pd.read_excel(r"C:\Users\julia.villela\Documents\Analista de Dados Sênior - Case.xlsx", sheet_name="Clientes")

In [6]:
df_contratos

,Código da Unidade,Código do Contrato,Contrato Matriz,Ano,Mês,Horas,Volume Contratado Antes da Sazonalização (MWh),Volume Mínimo p/ Sazonalização (MWh),Volume Máximo p/ Sazonalização (MWh),Percentual Mínimo para Flexibilidade,Percentual Máximo para Flexibilidade,Percentual de Atendimento do Consumo
0,000001-01,A00001,A00003,2022,1,744,0.0,0.0,0.0,0.0,0.0,1.0
1,000001-01,A00001,A00003,2022,2,672,0.0,0.0,0.0,0.0,0.0,1.0
2,000001-01,A00001,A00003,2022,3,744,0.0,0.0,0.0,0.0,0.0,1.0
3,000001-01,A00001,A00003,2022,4,720,0.0,0.0,0.0,0.0,0.0,1.0
4,000001-01,A00001,A00003,2022,5,744,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4051,000010-06,A00169,A00169,2027,8,744,4092.0,3682.8,4501.2,0.3,0.3,1.0
4052,000010-06,A00169,A00169,2027,9,720,3960.0,3564.0,4356.0,0.3,0.3,1.0
4053,000010-06,A00169,A00169,2027,10,744,4092.0,3682.8,4501.2,0.3,0.3,1.0
4054,000010-06,A00169,A00169,2027,11,720,3960.0,3564.0,4356.0,0.3,0.3,1.0


In [ ]:
#tabela de horas totais para cada ano da tabela de Contratos
df_horas_ano = df_contratos[['Ano','Mês','Horas']]
df_horas_ano = df_horas_ano.drop_duplicates()
df_horas_ano = df_horas_ano.groupby(['Ano']).agg(horas_ano = pd.NamedAgg(column = "Horas", aggfunc = "sum"))

In [ ]:
df_contratos = df_contratos.merge(df_consumo,how="left",on=['Código da Unidade','Mês'])
# Definindo a necessidade contratual de cada unidade, a cada mês, como seu consumo x o percentual de atendimento
df_contratos['necessidade'] = df_contratos['Consumo Total (MWh)']*df_contratos['Percentual de Atendimento do Consumo']


In [ ]:
# Agrupando por contrato matriz para obter os valores dos contratos

df_matriz = df_contratos.groupby(['Contrato Matriz','Ano','Mês']).agg(
                            total_contratos_mensal = pd.NamedAgg(column = "Volume Contratado Antes da Sazonalização (MWh)", aggfunc = "sum"),
                            sazo_min = pd.NamedAgg(column = "Volume Mínimo p/ Sazonalização (MWh)", aggfunc = "sum"),
                            sazo_max = pd.NamedAgg(column = "Volume Máximo p/ Sazonalização (MWh)", aggfunc = "sum"),
                            necessidade_matriz = pd.NamedAgg(column = "necessidade", aggfunc = "sum"))



In [ ]:
# Comparando a necessidade agregada da matriz aos seus volumes máximos e mínimos para cada mês
# Se a necessidade for menor do que a sazo mínima, haverá excedente (positivo)
df_matriz['exc_sazo'] = df_matriz['sazo_min'] - df_matriz['necessidade_matriz']
df_matriz['zero'] = 0

# Filtrando apenas os valores positivos (ou nulos), pois se for negativo, a função máximo vai considerar o zero
df_matriz['exc_sazo'] = df_matriz[['exc_sazo','zero']].max(axis=1)

# Se a necessidade agregada da matriz for maior que a sazo máxima, haverá exposição (negativo)
df_matriz['exp_sazo'] = df_matriz['sazo_max'] - df_matriz['necessidade_matriz']

# Filtrando apenas os valores negativos (ou nulos), pois se for positivo a função mínimo vai considerar o zero
df_matriz['exp_sazo'] = df_matriz[['exp_sazo','zero']].min(axis=1)

# Exposição e Excedente podem se somar pois se um não é nulo, o outro será (não existem ao mesmo tempo)
# O saldo então é a exposição (negativo) ou o excedente (positivo)
# Se a necessidade estiver entre as sazo max e min, o saldo será nulo
df_matriz['saldo_sazo_mensal'] = df_matriz['exc_sazo'] + df_matriz['exp_sazo']

# Definindo uma primeira proposta de sazonalização agregada da matriz, a sazo_aux
# Quando a necessidade agregada da matriz está fora do intervalo aceitável da sazo, a sazo_aux é a extremidade mais próxima
# Quando a necessidade está dentro deste intervalo, a sazo_aux é a própria necessidade agregada da matriz
df_matriz['sazo_aux']= df_matriz['necessidade_matriz'] + df_matriz['saldo_sazo_mensal']

# Nos casos em que a sazo_aux ficou no intervalo aceitável, é possível aumentar ou diminuir seu valor até chegar nos extremos
# disponivel_receber_mensal é o "espaço" entre a sazo_aux e a sazo max, o que ainda cabe naquele mês
df_matriz['disponivel_receber_mensal'] = df_matriz['sazo_max'] + df_matriz['sazo_aux']

# disponivel_retirar_mensal é o "crédito" entre a sazo_aux e a sazo min, o que ainda pode ser tirado daquele mês
df_matriz['disponivel_retirar_mensal'] = df_matriz['sazo_min'] + df_matriz['sazo_aux']
df_matriz = df_matriz.reset_index('Mês')

In [ ]:
# Agregando por ano, pois os contratos sazonalizados tem que respeitar o montante anual

df_matriz_anual = df_matriz.groupby(['Contrato Matriz','Ano']).agg(
                            total_contratos_anual = pd.NamedAgg(column = "total_contratos_mensal", aggfunc = "sum"),
                            sazo_aux_anual = pd.NamedAgg(column = "sazo_aux", aggfunc = "sum"),
                            disponivel_receber_anual = pd.NamedAgg(column = "disponivel_receber_mensal", aggfunc = "sum"),
                            disponivel_retirar_anual = pd.NamedAgg(column = "disponivel_retirar_mensal", aggfunc = "sum"))

# O ajuste_anual_a_distribuir é a diferença entre o montante anual antes da sazonalização e a a primeira proposta de sazo
# Se for positivo, é porque há mais energia contratada do que a proposta inicial de sazo, então é preciso aumentar a sazo
# Se for negativo, a proposta de sazo foi muit alta e será preciso diminuir os valores mensais
df_matriz_anual['ajuste_anual_a_distribuir'] = df_matriz_anual['total_contratos_anual'] - df_matriz_anual['sazo_aux_anual']
df_matriz_anual = df_matriz_anual.drop(['total_contratos_anual','sazo_aux_anual'], axis = 1)


In [ ]:
#Voltando a visão mensal, agregada por contratos matriz
df_matriz = df_matriz.merge(df_matriz_anual, how = "left", on = ['Contrato Matriz','Ano'])

# Agora temos que redistribuir o valor de ajuste (negativo ou positivo) entre os meses para ocontrato matriz

# Determinando quanto será tirado ou colocado em cada mês docontrato matriz
# A razao_receber é a proporção do "espaço" disponível em cada mês em relação a todo espaço disponível no contrato matriz
df_matriz['razao_receber'] = df_matriz['disponivel_receber_mensal'] / df_matriz['disponivel_receber_anual']

# Caso o espaço total do contrato seja zero, o resultado da divisão acima será not a number
# Substituindo nan por zero, pois se não há espaço no contrato total, não haverá em nenhum de seus meses
df_matriz['razao_receber'] = df_matriz['razao_receber'].fillna(0)

# A razao_retirar é a proporção do "crédito" disponível que pode ser retirado de cada mês em relação ao contrato matriz total
df_matriz['razao_retirar'] = df_matriz['disponivel_retirar_mensal'] / df_matriz['disponivel_retirar_anual']

# Caso o crédito total do contrato seja zero, o resultado da divisão acima será not a number
# Substituindo nan por zero, pois se não há crédito no contrato total, não haverá em nenhum de seus meses
df_matriz['razao_retirar'] = df_matriz['razao_receber'].fillna(0)

# Um mesmo mês pode ter valores não nulos de razao_recer e razão_retirar ao mesmo tempo
# Para não multiplicar uma razao_receber por um ajuste negativo (ou razao_retirar por ajuste negativo)
# Separando os ajustes positivos e negativos
df_matriz['ajuste_anual_positivo'] = df_matriz[['ajuste_anual_a_distribuir','zero']].max(axis=1)
df_matriz['ajuste_anual_negativo'] = df_matriz[['ajuste_anual_a_distribuir','zero']].min(axis=1)

# Multiplicando as razões pelos seus respectivos ajustes (positivo <-> receber e negativo <-> retirar)
df_matriz['receber_mensal'] = df_matriz['razao_receber'] * df_matriz['ajuste_anual_positivo']
df_matriz['retirar_mensal'] = df_matriz['razao_retirar'] * df_matriz['ajuste_anual_negativo']

# Como cada razão é uma parte que compõe o contrato matriz, a soma dos ajustes mensais será igual ao ajuste anual
# Somando os valores a receber e a retirar pois só um deles será não nulo
df_matriz['ajuste_mensal'] = df_matriz['receber_mensal'] + df_matriz['retirar_mensal']

# A nova proposta de sazo, então, respeita o valor total anual
df_matriz['sazo_mensal_matriz'] = df_matriz['ajuste_mensal'] + df_matriz['sazo_aux']

# Mantendo apenas as colunas que serão utilizadas
# A Coluna Ano não precisa ser listada pois é um índice, uma vez que foi usada para mesclar dataframes
df_matriz = df_matriz[
    ['Mês',
     'sazo_mensal_matriz',
     'necessidade_matriz',
     'zero']]


In [ ]:
#Tirando as colunas de valores (que são vazias para contratos dependentes) para não ficarem repetidas
df_contratos = df_contratos.drop(
    ['Volume Contratado Antes da Sazonalização (MWh)',
     'Volume Mínimo p/ Sazonalização (MWh)',
     'Volume Máximo p/ Sazonalização (MWh)'], axis = 1)


# Trazendo para cada contrato as informações calculadas da sua matriz
df_contratos = df_contratos.merge(df_matriz, how = "left", on = ['Contrato Matriz','Ano','Mês'])

# Distribuindo a sazonalização da matriz para seus contratos dependentes
# Cada contrato tem uma necessidade, baseada no consumo da unidade e seu percentual de atendimento
# Essa necessidade é uma parte que compõe a necessidade agregada da matriz, que fez parte dos cálculos até agora
# A razao_necessidade é a proporção da necessidade de um contrato em relação a soma de todos os dependentes da mesma matriz (para um mês)
df_contratos['razao_necessidade'] = df_contratos['necessidade'] / df_contratos['necessidade_matriz']

# Aplicando essa proporção para distribuir a sazonalização da matriz entre seus dependentes
df_contratos['sazo_mensal_unidade'] = df_contratos['razao_necessidade'] * df_contratos['sazo_mensal_matriz']


In [ ]:
# A sazonalização aplicada a cada mês é seu contrato flat
# Calculando os limites de flexibilidade por contrato dependente
df_contratos['flex_min_unidade'] = (1 - df_contratos['Percentual Mínimo para Flexibilidade'] ) * df_contratos['sazo_mensal_unidade']
df_contratos['flex_max_unidade'] = (1 + df_contratos['Percentual Máximo para Flexibilidade'] ) * df_contratos['sazo_mensal_unidade']

# Se a necessidade ficar fora dos limites da flex, haverá excedente (+) ou exposição (-)
df_contratos['excedente_mwh'] = df_contratos['flex_min_unidade'] - df_contratos['necessidade']
df_contratos['excedente_mwh'] = df_contratos[['excedente_mwh','zero']].max(axis=1)
df_contratos['exposicao_mwh'] = df_contratos['flex_max_unidade'] - df_contratos['necessidade']
df_contratos['exposicao_mwh'] = df_contratos[['exposicao_mwh','zero']].min(axis=1)

# Essas duas coisas não acontecem ao mesmo tempo, então podem ser somadas, pois somente um pode ser não nulo
df_contratos['saldo_mwh'] = df_contratos['excedente_mwh'] + df_contratos['exposicao_mwh']

# A necessidade "corrigida" pelo saldo (quando houver) fica sempre em um dos extremos da flex
# Caso não haja saldo, a flex exercida é a própria necessidade
df_contratos['flex_exercida_contrato_unidade_mensal_mwh'] = df_contratos['necessidade'] + df_contratos['saldo_mwh']

# Uma unidade pode ter mais de um contrato para suprir seu consumo
# Deixando de agrupar por contrato, passando para uma visão de unidade e somando a energia obtida em contratos para cada mês
df_contratos = df_contratos.groupby(['Código da Unidade', 'Ano', 'Mês']).agg(
        flex_exercida_unidade_mensal_mwh = pd.NamedAgg(column = "flex_exercida_contrato_unidade_mensal_mwh", aggfunc = "sum"),
        zero = pd.NamedAgg(column = "zero", aggfunc = "sum"))

In [ ]:
# Não podemos calcular o gap das unidades apenas a partir de uma tabela de contratos
# pois se se alguma unidade não tem contrato para um ano, seu consumo também não será considerado

# Fabricando uma tabela de base para o gap, considerando o ano atual e os próximos 5, com os dados de consumo mensal por unidade

este_ano = datetime.today().year
anos = np.array([este_ano, este_ano + 1, este_ano + 2, este_ano + 3, este_ano + 4, este_ano + 5])

for ano in anos:
    df_calendario_aux = df_consumo.assign(Ano = ano)
    if ano == anos[0]:
        df_gap_mensal = df_calendario_aux
    else:
        df_gap_mensal = pd.concat([df_gap_mensal, df_calendario_aux])

df_gap_mensal = df_gap_mensal.set_index('Ano')




In [ ]:
#Alimentando essa tabela de gap com a previsão de flex exercida para os meses em que as unidades têm contrato(s)
df_gap_mensal = df_gap_mensal.merge(df_contratos, how= "left", on = ['Código da Unidade', "Ano", "Mês"])

# Para os meses em que não há contrato, o campos novos serão nulos
# Para ser considerado nos cálculos é preciso substituir por zero
df_gap_mensal = df_gap_mensal.fillna(0)

#Para cada unidade, em cada mês, verificamos o saldo
df_gap_mensal['saldo_mensal'] = df_gap_mensal['flex_exercida_unidade_mensal_mwh'] - df_gap_mensal['Consumo Total (MWh)']
df_gap_mensal['excedente_mensal'] = df_gap_mensal[['saldo_mensal','zero']].max(axis=1)
df_gap_mensal['exposicao_mensal'] = df_gap_mensal[['saldo_mensal','zero']].min(axis=1)



In [ ]:
df_gap_anual = df_gap_mensal.groupby(['Código da Unidade','Ano']).agg(
                consumo_mwh = pd.NamedAgg(column = "Consumo Total (MWh)", aggfunc = "sum"),
                contrato_mwh = pd.NamedAgg(column = "flex_exercida_unidade_mensal_mwh", aggfunc = "sum"),
                excedente_mwh = pd.NamedAgg(column = "excedente_mensal", aggfunc = "sum"),
                exposicao_mwh = pd.NamedAgg(column = "exposicao_mensal", aggfunc = "sum"))

# A coluna Código da Unidade foi promovida a índice ao ser usada como chave para mesclar dfs
# Se o índice não é usado como chave na próxima operação de mesclar, a coluna inteira será descartada
# Então é preciso remover a coluna dos índices para mantê-la no df
df_gap_anual = df_gap_anual.reset_index('Código da Unidade')

# Buscando as horas de cada ano para converter em MWm
df_gap_anual = df_gap_anual.merge(df_horas_ano, how = "left", on = ['Ano'])
df_gap_anual['consumo_mwm'] = df_gap_anual['consumo_mwh'] / df_gap_anual['horas_ano']
df_gap_anual['contrato_mwm'] = df_gap_anual['contrato_mwh'] / df_gap_anual['horas_ano']
df_gap_anual['excedente_mwm'] = df_gap_anual['excedente_mwh'] / df_gap_anual['horas_ano']
df_gap_anual['exposicao_mwm'] = df_gap_anual['exposicao_mwh'] / df_gap_anual['horas_ano']



In [ ]:
df_gap_consolidado = df_gap_anual.groupby(['Ano']).agg(
                    consumo_consolidado = pd.NamedAgg(column = "consumo_mwm", aggfunc = "sum"),
                    contrato_consolidado = pd.NamedAgg(column = "contrato_mwm", aggfunc = "sum"),
                    excedente_consolidado = pd.NamedAgg(column = "excedente_mwm", aggfunc = "sum"),
                    exposicao_consolidado = pd.NamedAgg(column = "exposicao_mwm", aggfunc = "sum"))
df_gap_consolidado['percentual_excedente'] = df_gap_consolidado['excedente_consolidado'] / df_gap_consolidado['consumo_consolidado']
df_gap_consolidado['percentual_exposicao'] = df_gap_consolidado['exposicao_consolidado'] / df_gap_consolidado['consumo_consolidado']


In [ ]:
with pd.ExcelWriter('output_gap_sazo.xlsx') as writer:
    df_gap_anual.to_excel(writer, sheet_name='GAP')
    df_gap_consolidado.to_excel(writer, sheet_name='GAP Anual Consolidado')

print('Dados exportados com sucesso')